In [3]:
import os
from unstructured.partition.auto import partition

pdf_path = os.path.expanduser("~/pdf_to_rag/assets/RAG_em_CienciaDeDados.pdf")
elements = partition(pdf_path)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
paragraphs = [el for el in elements if el.category == "NarrativeText"]
for i, paragraph in enumerate(paragraphs[:5]):
    print(f"Paragraph {i+1}:\n{paragraph.text}\n")

Paragraph 1:
Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.

Paragraph 2:
Vamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.

Paragraph 3:
Por que o RAG é importante para a Ciência de Dados e Negócios? A principal vantagem do RAG é que ele oferece respostas mais precisas e contextualizadas ao extrair informações relevant

In [5]:
print(vars(paragraphs[0]))

{'text': 'Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.', 'embeddings': None, '_element_id': '0a69dbb5a1ad8d51199d3a77641ef2f5', 'metadata': <unstructured.documents.elements.ElementMetadata object at 0x17fb5b830>}


In [6]:
vars_paragraph = {
    'text': 'Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.',
    'embeddings': None,
    '_element_id': '0a69dbb5a1ad8d51199d3a77641ef2f5',
    'metadata': "<unstructured.documents.elements.ElementMetadata object at 0x1403c1010>"
}

In [7]:
metadata = paragraphs[0].metadata
print(vars(metadata))

{'coordinates': CoordinatesMetadata(points=((36.0, 94.24400000000003), (36.0, 123.28399999999999), (561.82, 123.28399999999999), (561.82, 94.24400000000003)), system=<unstructured.documents.coordinates.PixelSpace object at 0x17f0cf8c0>), 'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets', 'filename': 'RAG_em_CienciaDeDados.pdf', 'last_modified': '2025-08-06T08:22:22', 'links': [], 'page_number': 1, 'languages': ['por'], '_known_field_names': frozenset({'orig_elements', 'page_name', 'emphasized_text_contents', 'languages', 'emphasized_text_tags', 'category_depth', 'image_path', 'link_start_indexes', 'sent_to', 'header_footer_type', 'text_as_html', 'coordinates', 'link_urls', 'is_continuation', 'last_modified', 'subject', 'data_source', 'key_value_pairs', 'cc_recipient', 'image_url', 'image_base64', 'signature', 'url', 'table_as_cells', 'bcc_recipient', 'parent_id', 'page_number', 'links', 'detection_origin', 'file_directory', 'attached_to_filename', 'detection_class_prob', 'file

In [8]:
metadata_paragraph = {
    'coordinates': "CoordinatesMetadata(points=((36.0, 94.24400000000003), (36.0, 123.28399999999999), (561.82, 123.28399999999999), (561.82, 94.24400000000003)), system=<unstructured.documents.coordinates.PixelSpace object at 0x1402b4e90>)",
    'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets',
    'filename': 'RAG_em_CienciaDeDados.pdf',
    'last_modified': '2025-08-06T08:22:22',
    'links': [],
    'page_number': 1,
    'languages': ['por'],
    '_known_field_names': frozenset({'detection_class_prob', 'header_footer_type', 'text_as_html', 'filetype', 'page_name', 'links', 'image_path', 'image_mime_type', 'link_start_indexes', 'sent_from', 'link_urls', 'cc_recipient', 'subject', 'signature', 'is_continuation', 'filename', 'emphasized_text_contents', 'url', 'data_source', 'sent_to', 'last_modified', 'email_message_id', 'parent_id', 'languages', 'coordinates', 'category_depth', 'emphasized_text_tags', 'image_url', 'key_value_pairs', 'table_as_cells', 'file_directory', 'page_number', 'image_base64', 'detection_origin', 'orig_elements', 'attached_to_filename', 'link_texts', 'bcc_recipient'}),
    'filetype': 'application/pdf',
    'parent_id': '01ca41a6dcf3acc477735e9f013083f2'
}

In [9]:
from pydantic import BaseModel

class MetadataModel(BaseModel):
    coordinates: str
    file_directory: str
    filename: str
    last_modified: str
    links: list
    page_number: int
    languages: list
    _known_field_names: frozenset
    filetype: str
    parent_id: str

class ParagraphModel(BaseModel):
    text: str
    embeddings: None
    _element_id: str
    metadata: MetadataModel

In [10]:
paragraph_models = []
for paragraph in paragraphs: 
    metadata_dict = vars(paragraph.metadata)

    if not isinstance(metadata_dict['coordinates'], str):
        metadata_dict['coordinates'] = str(metadata_dict['coordinates'])
    
    metadata_model = MetadataModel(**metadata_dict)
    
    paragraph_model = ParagraphModel(
        text=paragraph.text,
        embeddings=getattr(paragraph, "embeddings", None),
        _element_id=getattr(paragraph, "_element_id", ""),
        metadata=metadata_model
    )

    paragraph_models.append(paragraph_model)

In [11]:
print(f"Text: {paragraph_models[0].text}")
print(f"Page Number: {paragraph_models[0].metadata.page_number}")
print(f"File Directory: {paragraph_models[0].metadata.file_directory}")
print(f"Filename: {paragraph_models[0].metadata.filename}")
print(f"Languages: {paragraph_models[0].metadata.languages}")

Text: Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.
Page Number: 1
File Directory: /Users/ryan.oliveira/pdf_to_rag/assets
Filename: RAG_em_CienciaDeDados.pdf
Languages: ['por']


In [12]:
for paragraph in paragraph_models:
    print(f"Paragraph Text: {paragraph.text}")

Paragraph Text: Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.
Paragraph Text: Vamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.
Paragraph Text: Por que o RAG é importante para a Ciência de Dados e Negócios? A principal vantagem do RAG é que ele oferece respostas mais precisas e contextualizadas ao extrair informações r

In [13]:
titles = [el.text for el in elements if el.category == "Title"]
for i, title in enumerate(titles):
    print(f"Title {i+1}:\n{title}\n")
print(len(titles))

Title 1:
Olá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!

Title 2:
pode exigir grande esforço de manutenção.

Title 3:
natural requer uma combinação de técnicas de IA avançadas.

Title 4:
se tornar complexo.

Title 5:
Passo a Passo de Implementação de RAG

5


In [14]:
headers = [el for el in elements if el.category == "Header"]
for i, header in enumerate(headers):
    print(f"Header {i+1}:\n{header.text}\n")
print(len(headers))

Header 1:
RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados

Header 2:
utilizar o SQLite ou o PostgreSQL, que são bancos de dados gratuitos e de código aberto, ideais para armazenar e consultar dados de forma eficiente.

2


In [15]:
list_item = [el for el in elements if el.category == "ListItem"]
for i, item in enumerate(list_item):
    print(f"List Item {i+1}:\n{item.text}\n")
print(len(list_item))

List Item 1:
Curadoria de dados: É essencial que as bases de dados sejam precisas e atualizadas, o que

List Item 2:
Treinamento de modelos: Integrar recuperação de informações com modelos de linguagem

List Item 3:
Escalabilidade: Em grandes volumes de dados, manter a performance do sistema RAG pode

List Item 4:
1. Criação de uma base de dados de contratos antigos: Colete todos os contratos anteriores e armazene-os em um formato acessível (JSON, CSV, banco de dados). Para isso, você pode

List Item 5:
2. Desenvolvimento ou integração com um modelo de linguagem natural: Utilize um modelo de IA como o GPT, que será responsável por gerar novas versões dos contratos. Existem vários modelos de código aberto e gratuitos, como o Llama, Falcon, Mistral, Phi, Gemma entre outros. Essas ferramentas oferecem modelos de linguagem poderosos, acessíveis e escaláveis para microempresas.

List Item 6:
3. Configuração da camada de recuperação de dados: Configure um sistema de busca que permita recuper

In [27]:
from unstructured.chunking.basic import chunk_elements

chunks = chunk_elements(elements, max_characters=1000)
chunks_metadata = [vars(chunk.metadata) for chunk in chunks]

print(vars(chunks[1]))
print(vars(chunks[0].metadata))

{'text': 'Por que o RAG é importante para a Ciência de Dados e Negócios? A principal vantagem do RAG é que ele oferece respostas mais precisas e contextualizadas ao extrair informações relevantes de fontes específicas e usá-las para melhorar a geração de texto. Na área de ciência de dados, onde o volume de informações pode ser vasto e complexo, o RAG permite extrair insights que vão além do que um modelo tradicional de IA poderia alcançar sozinho. Para as empresas, isso representa um salto na qualidade das tomadas de decisão, já que a capacidade de recuperar informações com precisão permite que dados históricos e atuais sejam utilizados de maneira estratégica.\n\nEmbora o RAG ofereça muitas vantagens, ele também apresenta alguns desafios:\n\nCuradoria de dados: É essencial que as bases de dados sejam precisas e atualizadas, o que\n\npode exigir grande esforço de manutenção.\n\nTreinamento de modelos: Integrar recuperação de informações com modelos de linguagem', 'embeddings': None, '_e

In [26]:
{
    'text': 'RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados\n\nOlá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!\n\nNesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.\n\nVamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.', 
    'embeddings': None, 
    '_element_id': None, 
    'metadata': "<unstructured.documents.elements.ElementMetadata object at 0x17f09e5a0>}"
}


{'text': 'RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados\n\nOlá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!\n\nNesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.\n\nVamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.',
 'embeddings': None,
 '_element_id': None,
 'metadata': '<unstructure

In [17]:
{
    'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets', 
    'filename': 'RAG_em_CienciaDeDados.pdf', 
    'filetype': 'application/pdf', 
    'languages': ['por'], 
    'last_modified': '2025-08-06T08:22:22', 
    'page_number': 1, 
    'orig_elements': "[<unstructured.documents.elements.Header object at 0x13e5eff20>, <unstructured.documents.elements.Title object at 0x13dc84980>, <unstructured.documents.elements.NarrativeText object at 0x13e6aef90>]"
}

{'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets',
 'filename': 'RAG_em_CienciaDeDados.pdf',
 'filetype': 'application/pdf',
 'languages': ['por'],
 'last_modified': '2025-08-06T08:22:22',
 'page_number': 1,
 'orig_elements': '[<unstructured.documents.elements.Header object at 0x13e5eff20>, <unstructured.documents.elements.Title object at 0x13dc84980>, <unstructured.documents.elements.NarrativeText object at 0x13e6aef90>]'}

In [32]:
import pydantic

class ChunkMetadataModel(BaseModel):
    file_directory: str
    filename: str
    filetype: str
    languages: list
    last_modified: str
    page_number: int
    orig_elements: list

class ChunkModel(BaseModel):
    text: str
    embeddings: None
    _element_id: str
    metadata: ChunkMetadataModel

In [33]:
chunks_model = [
    ChunkModel(
        text=chunk.text,
        embeddings=None,
        _element_id=chunk._element_id if chunk._element_id is not None else "",
        metadata=ChunkMetadataModel(**vars(chunk.metadata))
    )
    for chunk in chunks
]

In [34]:
from sentence_transformers import SentenceTransformer


# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([chunk.text for chunk in chunks_model], convert_to_tensor=True).tolist()


In [36]:
print("Embeddings:")
for emb in embeddings:
    print(emb)

Embeddings:
[-0.06621938943862915, 0.05365533009171486, -0.017777094617486, -0.04171613231301308, -0.1044425442814827, 0.003414630191400647, -0.030998174101114273, 0.0222834050655365, 0.020139211788773537, 0.0038597530219703913, 0.0725453644990921, 0.03510088473558426, 0.04884802922606468, -0.01623886078596115, 0.004113408271223307, 0.030291087925434113, 0.04122956469655037, 0.12285242229700089, 0.008892645128071308, 0.03619732707738876, 0.08289770036935806, 0.029823094606399536, 0.01590513437986374, -0.03611769154667854, -0.00907345861196518, 0.023618631064891815, -0.06819714605808258, -0.007117491215467453, 0.0624820776283741, -0.023336654528975487, -0.030031023547053337, 0.1128462478518486, 0.055729229003190994, 0.02768901363015175, -0.07163132727146149, -0.007702064234763384, 0.011125879362225533, 0.023205533623695374, -0.009321282617747784, 0.06900843232870102, -0.08921454846858978, 0.05645354837179184, -0.011567621491849422, -0.09875094890594482, 0.06442739814519882, -0.016950756

In [37]:
# Storage embeddings

import chromadb

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")

InternalError: Collection [my_collection] already exists

In [41]:
def sanitize_metadata(metadata):
    sanitized = {}
    for k, v in metadata.items():
        if isinstance(v, list):
            sanitized[k] = ",".join(map(str, v))
        elif isinstance(v, frozenset):
            sanitized[k] = ",".join(map(str, v))
        else:
            sanitized[k] = v
    return sanitized

collection.add(
    documents=[chunk.text for chunk in chunks_model],
    embeddings=embeddings,
    metadatas=[sanitize_metadata(chunk.metadata.model_dump()) for chunk in chunks_model],
    ids=[str(chunk.metadata.orig_elements) for chunk in chunks_model]
)

In [42]:
# Consultando a coleção para verificar se os embeddings foram corretamente armazenados
result = collection.query(
    query_texts=["O que é RAG?"],
    n_results=2
)

/Users/ryan.oliveira/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:12<00:00, 6.84MiB/s]



In [51]:
print(result['documents'])

[['RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados\n\nOlá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!\n\nNesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.\n\nVamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.', 'natural requer uma combinação de técnicas de IA avançadas.\n\nEscalabilidad

In [45]:
{
    'ids': [['[<unstructured.documents.elements.Header object at 0x17f9fc9e0>, <unstructured.documents.elements.Title object at 0x17fac9f10>, <unstructured.documents.elements.NarrativeText object at 0x17fb5ac30>, <unstructured.documents.elements.NarrativeText object at 0x17f994aa0>]', '[<unstructured.documents.elements.Title object at 0x17e8081d0>, <unstructured.documents.elements.ListItem object at 0x17fb63500>, <unstructured.documents.elements.Title object at 0x17fb637d0>, <unstructured.documents.elements.NarrativeText object at 0x17fb6d0a0>, <unstructured.documents.elements.Title object at 0x17f9c7b30>]']], 
    'embeddings': None, 
    'documents': [
        ['RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados\n\nOlá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!\n\nNesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.\n\nVamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.', 
        'natural requer uma combinação de técnicas de IA avançadas.\n\nEscalabilidade: Em grandes volumes de dados, manter a performance do sistema RAG pode\n\nse tornar complexo.\n\nAlgumas das principais aplicações do RAG incluem: Automação de Atendimento ao Cliente: Fornece respostas rápidas e contextualizadas para dúvidas de clientes, baseado em informações recuperadas de bases de conhecimento internas; Geração de Documentos Jurídicos: Automação na criação de contratos, baseando-se em exemplos anteriores e gerando novos documentos personalizados; Análise de Dados: Extração de insights valiosos ao combinar dados históricos com novas tendências, proporcionando uma visão completa para tomada de decisões; entre outras.\n\nPasso a Passo de Implementação de RAG']
        ],
    'uris': None, 
    'included': ['metadatas', 'documents', 'distances'], 
    'data': None, 
    'metadatas': [[{'last_modified': '2025-08-06T08:22:22', 'filename': 'RAG_em_CienciaDeDados.pdf', 'languages': 'por', 'page_number': 1, 'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets', 'orig_elements': 'RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados,Olá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!,Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.,Vamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.', 'filetype': 'application/pdf'}, {'last_modified': '2025-08-06T08:22:22', 'orig_elements': 'natural requer uma combinação de técnicas de IA avançadas.,Escalabilidade: Em grandes volumes de dados, manter a performance do sistema RAG pode,se tornar complexo.,Algumas das principais aplicações do RAG incluem: Automação de Atendimento ao Cliente: Fornece respostas rápidas e contextualizadas para dúvidas de clientes, baseado em informações recuperadas de bases de conhecimento internas; Geração de Documentos Jurídicos: Automação na criação de contratos, baseando-se em exemplos anteriores e gerando novos documentos personalizados; Análise de Dados: Extração de insights valiosos ao combinar dados históricos com novas tendências, proporcionando uma visão completa para tomada de decisões; entre outras.,Passo a Passo de Implementação de RAG', 'page_number': 1, 'filename': 'RAG_em_CienciaDeDados.pdf', 'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets', 'filetype': 'application/pdf', 'languages': 'por'}]], 
    'distances': [[0.9171342849731445, 0.9288123250007629]]
}

{'ids': [['[<unstructured.documents.elements.Header object at 0x17f9fc9e0>, <unstructured.documents.elements.Title object at 0x17fac9f10>, <unstructured.documents.elements.NarrativeText object at 0x17fb5ac30>, <unstructured.documents.elements.NarrativeText object at 0x17f994aa0>]',
   '[<unstructured.documents.elements.Title object at 0x17e8081d0>, <unstructured.documents.elements.ListItem object at 0x17fb63500>, <unstructured.documents.elements.Title object at 0x17fb637d0>, <unstructured.documents.elements.NarrativeText object at 0x17fb6d0a0>, <unstructured.documents.elements.Title object at 0x17f9c7b30>]']],
 'embeddings': None,
 'documents': [['RAG (Retrieval Augmented Generation) Aplicado à Ciência de Dados\n\nOlá! Bem-vindo(a) à nossa newsletter semanal sobre Ciência de Dados!\n\nNesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.\n\nVamos começar pel